# 公式Tutorialの実行 (part 2)

## 分類器の追加

<b><a href="Watson_NLC_02.ipynb">前回レポート</a></b>と同じような手順で、新しい分類器を、NLCインスタンスに追加してみます。

### 認証

NaturalLanguageClassifierV1クラス生成時に、サービス作成時に発行された認証情報を設定します。

In [1]:
import json
from watson_developer_cloud import NaturalLanguageClassifierV1

natural_language_classifier = NaturalLanguageClassifierV1(
  username='7fdb13f0-1c16-48f5-9a7d-6374ee56925d',
  password='LLwyqghs3cPB')

list関数で、すでに作成されている分類器を確認します。

（<b><a href="Watson_NLC_02.ipynb">前回レポート</a></b>で作成された分類器が、リストに含まれています）

In [2]:
classifiers = natural_language_classifier.list()
classifiers

{'classifiers': [{'classifier_id': 'f5bbbbx174-nlc-4304',
   'created': '2017-02-18T01:16:40.434Z',
   'language': 'ja',
   'name': 'My Classfier',
   'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f5bbbbx174-nlc-4304'}]}

### 訓練データの作成

<b><a href="Watson_NLC_99.ipynb">こちら</a></b>で作成しました。14,916レコードになります。

（「test_daikin_conversation.csv」が元データ）

### 分類器作成

create関数によりAPIコールを実行します。

In [4]:
test_data_file = 'resources/train_daikin_conversation.csv'
f = open(test_data_file, 'rt') # これは確認用
print(f.read()[:100])
f.close()

"Outlook 2010 にて メール を 送信 する 際 、 自分 宛 の メール を BCC で 運用 し たい の です が ？ ",3397
"Outlook 2010 にて 、 メール の


In [5]:
classifier_id = None
status = None
with open(test_data_file, 'rb') as training_data:
  classifier = natural_language_classifier.create(
    training_data=training_data,
    name='Daikin Classfier',
    language='ja'
  )
  classifier_id = classifier['classifier_id']
  status = classifier['status']

WatsonException: Error: Malformed data, Description: One or more fields in the training data at line 109, Code: 400

エラーになってしまいました。１０９行目のフォーマットが不正とのことですので、チェックして修正します。

誤：フィールドの中に２重引用符が混入している
~~~
"日本語 ＯＳ を 使用 し て い ます が 、 IE 11 インストール 作業 中 に " This is for Japanese . To install please contact DKI Support ( 7 - 205 - 413 )" と 表示 さ れ まし た 。 ",3504

~~~
正：混入している２重引用符を１重引用符に変えてあげる
~~~
"日本語 ＯＳ を 使用 し て い ます が 、 IE 11 インストール 作業 中 に ' This is for Japanese . To install please contact DKI Support ( 7 - 205 - 413 )' と 表示 さ れ まし た 。 ",3504

~~~

新しいデータで再実行します。

In [10]:
classifier_id = None
status = None
with open(test_data_file, 'rb') as training_data:
  classifier = natural_language_classifier.create(
    training_data=training_data,
    name='Daikin Classfier',
    language='ja'
  )
  classifier_id = classifier['classifier_id']
  status = classifier['status']

### 分類器作成状況の照会

14,916レコードなので、相当な時間（１５０分ほど）を要します。

In [27]:
statusDict = natural_language_classifier.status('f5bbbbx174-nlc-4312')
statusDict

{'classifier_id': 'f5bbbbx174-nlc-4312',
 'created': '2017-02-18T03:09:34.319Z',
 'language': 'ja',
 'name': 'Daikin Classfier',
 'status': 'Available',
 'status_description': 'The classifier instance is now available and is ready to take classifier requests.',
 'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f5bbbbx174-nlc-4312'}

In [28]:
classifiers = natural_language_classifier.list()
classifiers

{'classifiers': [{'classifier_id': 'f5bbbbx174-nlc-4304',
   'created': '2017-02-18T01:16:40.434Z',
   'language': 'ja',
   'name': 'My Classfier',
   'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f5bbbbx174-nlc-4304'},
  {'classifier_id': 'f5bbbbx174-nlc-4312',
   'created': '2017-02-18T03:09:34.319Z',
   'language': 'ja',
   'name': 'Daikin Classfier',
   'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f5bbbbx174-nlc-4312'}]}

### 分類実行

classify関数によりAPIコールを実行します。

In [29]:
from time import time

def predict_class(natural_language_classifier, idx, question_text):
    t0 = time()

    classifiers = natural_language_classifier.list()
    classifier_id = classifiers['classifiers'][idx]['classifier_id']
    
    classesDict = natural_language_classifier.classify(classifier_id, question_text)
    topclass = classesDict['classes'][0]

    print('class name=%s, confidence=%0.1f%%' % (topclass['class_name'], float(topclass['confidence'])*100.0))
    print("done in %0.3fs." % (time() - t0))

    return topclass

#### 質問試行 (1)

同じような意味で文脈が異なる質問文を使い、試してみました。

上は不正解（メールパスワード変更に関する質問の回答）、下は正解となりました。

In [32]:
question_text = 'メールに署名する方法を教えてください'
top_class = predict_class(natural_language_classifier, 1, question_text)

class name=3451, confidence=46.2%
done in 2.874s.


【ご参考：回答】以下URLに接続しメールのパスワードを更新してください。（以下略）

In [31]:
question_text = '署名入りのメールを送る方法を教えてください'
top_class = predict_class(natural_language_classifier, 1, question_text)

class name=3398, confidence=51.7%
done in 3.051s.


【ご参考：回答】署名の作り方については『【Outlook2010操作マニュアル 応用編】 署名の作成』を参照してください。（以下略）

#### 違った質問で再試行 (2)

テストデータにない「Galaxy S6」という単語を使って質問します。正解ではないのですが、それなりに近い予測結果になっています。

In [35]:
question_text = 'Galaxy S6のOSアップデート方法を教えていただけないでしょうか？'
top_class = predict_class(natural_language_classifier, 1, question_text)

class name=3809, confidence=69.5%
done in 4.500s.


【ご参考：回答】iOSのアップデート方法は2つあります。(1)Wi-Fi経由でのアップデート(2)iTunesがインストールされたパソコン経由でのアップデート
（以下省略）

#### 違った質問で再試行 (3)

テストデータとは全く無関係の質問をします。

結果は、無関係なクラス（Adobe Readerアップデータ設定に関する質問の回答）が戻ってきます。

ただし confidence はかなり低いです。

In [34]:
question_text = '今日の天気を教えていただけますでしょうか？'
top_class = predict_class(natural_language_classifier, 1, question_text)

class name=3670, confidence=22.4%
done in 2.802s.


【ご参考：回答】AdobeReadeのアップデータの設定は自動アップデートしない設定にしてください。以下手順を実施ください。（以下略）